<a id='Centre-Left-Scraping-and-cleaning'></a>

In [1]:
import requests
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#from langdetect import detect

In [2]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

In [3]:
np.set_printoptions(threshold=np.inf)
pd.options.display.max_colwidth = 1000

- List of the people I considered for this reasearch

In [28]:
lista = ['Bill Moyers', 
         'Anthony Giddens', 
         'Paul Krugman',
         'Benjamin Barber', 
         'Joseph E. Stiglitz',
         'Paul Mason',
         'Owen Jones', 
         'Jesse Jackson',
         'George Soros', 
         'Donna Brazile', 
         'James Carville',
         'Robert Reich',
         'Tony Judt', 
         'Jane Jacobs', 
         'Lawrence Lessig',
         'Simon Mckay',
         'Glenn Greenwald',
         'Ezra Klein',
         'Nicholas Kristof', 
         'Juan Williams',
         'Al Gore',
         'E. J. Dionne Jr.',
         'Alan Bennett',
         'Polly Toynbee'
         
]

- Getting Data from Goodreads

In [29]:
#List of name in the format for Goodreads website
new_lista = []
temp_lista = []

for name in lista:
    temp_lista.append(name.replace('.',''))


for n in temp_lista:
    new_lista.append(n.replace(' ','+'))
    

In [30]:
new_lista

['Bill+Moyers',
 'Anthony+Giddens',
 'Paul+Krugman',
 'Benjamin+Barber',
 'Joseph+E+Stiglitz',
 'Paul+Mason',
 'Owen+Jones',
 'Jesse+Jackson',
 'George+Soros',
 'Donna+Brazile',
 'James+Carville',
 'Robert+Reich',
 'Tony+Judt',
 'Jane+Jacobs',
 'Lawrence+Lessig',
 'Simon+Mckay',
 'Glenn+Greenwald',
 'Ezra+Klein',
 'Nicholas+Kristof',
 'Juan+Williams',
 'Al+Gore',
 'E+J+Dionne+Jr',
 'Alan+Bennett',
 'Polly+Toynbee']

In [31]:
#Creating a list for the Brainquote format and for a Goodreads Double check
brain_lista = []
for n in lista:
    brain_lista.append(n.lower().replace(' ','_'))


In [174]:
#Scraping all the existing quotes for the person I need from Goodreads
quotes = []
author = []

for name in new_lista:
    second_temp_quote = []
    second_temp_author = []
    r = requests.get("https://goodquotesapi.herokuapp.com/author/{}".format(name))
    data = r.json()
    if len(data) < 1:
        pass
    else:
        for page in tqdm_notebook(range(1,data['total_pages']+1)):
            new_r = requests.get("https://goodquotesapi.herokuapp.com/author/{}?page={}".format(name,page))
            new_data = new_r.json()
            first_temp_quote = []
            first_temp_author = []

            for i in range(20):
                temp_quote = []
                temp_author = []

                try: 
                    temp_quote.append(new_data['quotes'][i]['quote'])
                    temp_author.append(new_data['quotes'][0]['author'])

                except:
                    temp_quote.append(np.nan)
                    temp_author.append(np.nan)
            first_temp_author.append(temp_author)
            first_temp_quote.append(temp_quote)
            #second_temp_quote.append(first_temp_quote)
            #second_temp_author.append(first_temp_author)
        quotes.append(first_temp_quote)
        author.append(first_temp_author)

In [175]:
#Creating the goodreads Dataframe
goodreads = pd.DataFrame({
    'Author': [item for l in author for sublist in l  for item in sublist], 
    'Quotes': [item for l in quotes for sublist in l  for item in sublist],
    'source': 'goodreads'
})

In [176]:
goodreads.shape

(17, 3)

In [35]:
#Leaving every cell in a string
goodreads.Author = goodreads.Author.apply(lambda x : x[0])
goodreads.Quotes = goodreads.Quotes.apply(lambda x : x[0])

In [36]:
#Deleting all the rows where the Author is not in the list
goodreads = goodreads[goodreads['Author'].isin(lista)]

- Getting data from Wikiquote // I took the data in a different Jupyter Notebook

[Wiki scraping / cleaning](../WikiQuote/Centre_Left.ipynb#Wiki-Scraping-and-cleaning)

In [47]:
#Loading the previous dataframe scraped from Wikiquotes
wikiquotes = pd.read_csv('../WikiQuote/centre_left.csv')

In [48]:
#Checking the author
wikiquotes.Author.value_counts()

Paul Krugman               98
George Soros               80
Lawrence Lessig            56
Tony Judt                  33
Bill Moyers                27
George Will                26
Jane Jacobs                25
Joseph Stiglitz            23
James Carville             22
Benjamin Barber            15
Anthony Giddens            13
Paul Mason (journalist)    11
Jesse Jackson               9
Robert Reich                4
Donna Brazile               3
Owen Jones (writer)         1
Name: Author, dtype: int64

In [49]:
#removing few Authors
wiki_mask = (wikiquotes['Author'] != 'Owen Jones (writer)') & (wikiquotes['Author'] != 'Paul Mason (journalist)') & (wikiquotes['Author'] != 'George Will')


In [50]:
wikiquotes=wikiquotes[wiki_mask]

In [51]:
wikiquotes.Author.value_counts()

Paul Krugman       98
George Soros       80
Lawrence Lessig    56
Tony Judt          33
Bill Moyers        27
Jane Jacobs        25
Joseph Stiglitz    23
James Carville     22
Benjamin Barber    15
Anthony Giddens    13
Jesse Jackson       9
Robert Reich        4
Donna Brazile       3
Name: Author, dtype: int64

In [52]:

wikiquotes = wikiquotes.drop('Political Orientation', axis=1)

In [53]:
wikiquotes['source'] = 'wikiquotes'

 - Getting data from Brainquote  

In [54]:
#Scraping the quotes from Brainquote website
q = []
a = []

for name in brain_lista:
    temp_quote = []
    temp_author = []
    r = requests.get("https://www.brainyquote.com/authors/{}".format(name))
    soup = BeautifulSoup(r.text,'html.parser')
    for each in soup.find_all('a', attrs={'title':"view quote"}):
        temp_quote.append(each.text)
        #if len(temp_quote) > 0:
        temp_author.append(name)
    q.append(temp_quote)
    a.append(temp_author)
    


In [55]:
#Creating the Brainquote dataframe
brain = pd.DataFrame({'Author': [item for l in a for item in l],
                      'Quotes': [item for l in q for item in l],
                      'source': 'brainquote'})

In [56]:
#Removing '_' from the name and capitalize Name and Surname
brain.Author = brain.Author.apply(lambda x : x.replace('_',' ').title())

In [108]:
#Putting everything together
Centre_left_quotes = pd.concat([goodreads, brain, wikiquotes, d_goodreads])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [109]:
Centre_left_quotes = Centre_left_quotes.drop_duplicates(subset='Quotes')

In [110]:
Centre_left_quotes['language'] = Centre_left_quotes.Quotes.apply(lambda x : detect(x))

- Deleting all the not english quotes

In [111]:
Centre_left_quotes.language.value_counts()

en    2048
pt      26
es      19
it      19
cs      14
bg       9
ar       8
de       6
fi       4
nl       3
fr       3
no       1
da       1
tl       1
pl       1
Name: language, dtype: int64

In [112]:
Centre_left_quotes = Centre_left_quotes[Centre_left_quotes.language == 'en']

In [113]:
Centre_left_quotes.Author.value_counts()

Tony Judt             334
Glenn Greenwald       243
Alan Bennett          173
Paul Krugman          165
George Soros          142
Al Gore               136
Jane Jacobs           131
Joseph E. Stiglitz    127
Lawrence Lessig       119
Donna Brazile         108
Jesse Jackson          69
James Carville         67
Bill Moyers            64
Robert Reich           30
Juan Williams          26
Nicholas Kristof       26
Polly Toynbee          24
Joseph Stiglitz        23
Benjamin Barber        15
Ezra Klein             13
Anthony Giddens        13
Name: Author, dtype: int64

In [114]:
#Correcting the string name of same authors
Centre_left_quotes.loc[Centre_left_quotes['Author'].str.contains('Stiglitz'), 'Author'] = 'Joseph E. Stiglitz'

- Checking for quote in the quotes

In [115]:
dup = {}
#count = 0
for name in Centre_left_quotes.Author.unique():
    conta = []
    for text in Centre_left_quotes[Centre_left_quotes.Author==name].Quotes :
        count = 0
        for i in range(len(Centre_left_quotes[Centre_left_quotes.Author==name])):
            if text in Centre_left_quotes[Centre_left_quotes.Author==name].iloc[i,1]:
                count += 1


        conta.append(count)
    dup[name] = conta

In [146]:
Centre_left_quotes['Counter'] = 0

In [165]:
for name in Centre_left_quotes.Author.unique():
    print(name)
    Centre_left_quotes.loc[Centre_left_quotes.Author==name, 'Counter'] = dup[name]


Bill Moyers
Paul Krugman
Joseph E. Stiglitz
Jesse Jackson
George Soros
Donna Brazile
James Carville
Tony Judt
Jane Jacobs
Lawrence Lessig
Glenn Greenwald
Al Gore
Alan Bennett
Robert Reich
Ezra Klein
Nicholas Kristof
Juan Williams
Polly Toynbee
Anthony Giddens
Benjamin Barber


In [178]:
Centre_left_quotes.Counter.value_counts()

1    1925
2     106
3      11
4       5
5       1
Name: Counter, dtype: int64

In [185]:
Centre_left_quotes=Centre_left_quotes[Centre_left_quotes.Counter==1]

In [187]:
Centre_left_quotes.drop(['source', 'language', 'Counter'], axis=1, inplace=True)

- Adding the label

In [188]:
Centre_left_quotes['Political Orientation'] = 'Progressive'

In [214]:
rec = pd.read_csv('test_centre_left.csv')

In [217]:
CL = pd.concat([rec, Centre_left_quotes])

In [218]:
CL.Author.value_counts()

Tony Judt             300
Glenn Greenwald       214
Paul Krugman          165
Alan Bennett          164
Joseph E. Stiglitz    140
George Soros          136
Al Gore               136
Jane Jacobs           127
Donna Brazile         103
Lawrence Lessig        96
Jesse Jackson          69
James Carville         66
Bill Moyers            63
Robert Reich           30
Nicholas Kristof       26
Juan Williams          25
Polly Toynbee          24
Benjamin Barber        15
Ezra Klein             13
Anthony Giddens        13
Name: Author, dtype: int64

In [219]:
#Selecting 4 Authors for a total of bit more than 100 quotes for the final test df
list_test = ['Ezra Klein', 'Nicholas Kristof', 'Robert Reich', 'Polly Toynbee']

In [222]:
#Creating the Dataframe for the test
CL_test = CL[CL['Author'].isin(list_test)]


In [223]:
CL_test.Author.value_counts()

Robert Reich        30
Nicholas Kristof    26
Polly Toynbee       24
Ezra Klein          13
Name: Author, dtype: int64

In [224]:
#removing the 4 authors from the main database
Centre_left_quotes = CL[~CL['Author'].isin(list_test)]

In [225]:
#checking everyhting is fine
Centre_left_quotes.Author.value_counts()

Tony Judt             300
Glenn Greenwald       214
Paul Krugman          165
Alan Bennett          164
Joseph E. Stiglitz    140
George Soros          136
Al Gore               136
Jane Jacobs           127
Donna Brazile         103
Lawrence Lessig        96
Jesse Jackson          69
James Carville         66
Bill Moyers            63
Juan Williams          25
Benjamin Barber        15
Anthony Giddens        13
Name: Author, dtype: int64

In [5]:
#csv file for the train
Centre_left_quotes.to_csv('train_centre_left.csv', index=False)

In [227]:
#csv file for the test
CL_test.to_csv('test_centre_left.csv', index=False)

In [228]:
CL.to_csv('centre_left_quotes.csv')

In [229]:
CL_test.Author.value_counts()

Robert Reich        30
Nicholas Kristof    26
Polly Toynbee       24
Ezra Klein          13
Name: Author, dtype: int64

**SECOND PART**

In [5]:
#Loading files


In [82]:
CL_train = pd.read_csv('train_centre_left.csv')
CL_test = pd.read_csv('test_centre_left.csv')

In [83]:
CL_train.drop(CL_train.index[[1223, 91, 93, 278, 1232, 306, 338, 466, 
                                                904, 951, 1119, 1052, 1157, 1135, 1498, 
                                                228, 278,223, 238, 247, 69, 404, 368, 
                                                341, 329, 453, 476,218, 1202, 1129, 1160,
                                                584,1610, 1688, 628, 950, 936, 949, 818,  
                                                1058, 1090, 1031, 1024, 72, 653, 853, 844,
                                               847, 767]], inplace=True)

- Looking for quotes about the authors and not from them

In [84]:
def find_autoref(df):
    autoref = {}
    for name in df.Author.unique():
        conta = []

        for text in df[df.Author==name].Quotes :
            count = 0

            if name in text or name.split(' ')[-1] in text:
                    count += 1

            conta.append(count)
        autoref[name] = conta 
    return autoref

In [85]:
autoref = find_autoref(CL_train)

In [86]:
CL_train['auto'] = 0
for name in CL_train.Author.unique():
    print(name, ' done')
    CL_train.loc[CL_train.Author==name, 'auto'] = autoref[name]

Juan Williams  done
Bill Moyers  done
Paul Krugman  done
Joseph E. Stiglitz  done
Jesse Jackson  done
George Soros  done
Donna Brazile  done
James Carville  done
Tony Judt  done
Jane Jacobs  done
Lawrence Lessig  done
Glenn Greenwald  done
Al Gore  done
Alan Bennett  done
Anthony Giddens  done
Benjamin Barber  done


In [88]:
CL_train = CL_train[CL_train.auto==0]

- Checking for quotes in the quote with tokenize quotes

In [89]:
from textacy.preprocess import preprocess_text
import re
#import nltk

In [90]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(CL_train)

,Author,Quotes,Contain,Political Orientation,auto


In [91]:
CL_train['texicity'] = CL_train.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True, 
                                                                        no_numbers=True))

In [92]:
CL_train.head()

,Author,Quotes,Contain,Political Orientation,auto,texicity
0,Juan Williams,"In the 2000 presidential election, Al Gore got more votes than George W. Bush, but still lost the election. The Supreme Court's ruling in Florida gave Bush that pivotal state, and doomed Gore to lose the Electoral College. That odd scenario - where the candidate with the most votes loses - has happened three times in U.S. history.",no,Progressive,0,in the number presidential election al gore got more votes than george w bush but still lost the election the supreme court s ruling in florida gave bush that pivotal state and doomed gore to lose the electoral college that odd scenario where the candidate with the most votes loses has happened three times in u s history
1,Juan Williams,"While Planned Parenthood provides abortions at some of their clinics, it also provides healthcare services for poor women, including checkups, mammograms, cervical cancer screenings and contraceptives.",no,Progressive,0,while planned parenthood provides abortions at some of their clinics it also provides healthcare services for poor women including checkups mammograms cervical cancer screenings and contraceptives
2,Juan Williams,"The 2012 presidential campaign's turn away from the classic, straight-up, American election - where the candidate who gets the most votes nationwide wins - is another sad reminder of the extreme political polarization distorting today's politics. No one talks about a 50-state strategy for winning the presidency these days.",no,Progressive,0,the number presidential campaign s turn away from the classic straight up american election where the candidate who gets the most votes nationwide wins is another sad reminder of the extreme political polarization distorting today s politics no one talks about a number state strategy for winning the presidency these days
3,Juan Williams,I support gun control. But speaking honestly about the combustible mix of race and guns may be more important to stopping the slaughter in minority communities than any new gun-control laws.,no,Progressive,0,i support gun control but speaking honestly about the combustible mix of race and guns may be more important to stopping the slaughter in minority communities than any new gun control laws
4,Juan Williams,I pointed out that the Atlanta Olympic bomber - as well as Timothy McVeigh and the people who protest against gay rights at military funerals - are Christians but we journalists don't identify them by their religion.,no,Progressive,0,i pointed out that the atlanta olympic bomber as well as timothy mcveigh and the people who protest against gay rights at military funerals are christians but we journalists do not identify them by their religion


In [94]:
stops = stopwords.words('english')

In [96]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stops])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    return df
    

In [97]:
CL_train = tok_stop(CL_train)

In [98]:
fake = {}
#count = 0
for name in CL_train.Author.unique():
    print(name)
    conta = []
    for j,text in enumerate(CL_train[CL_train.Author==name].texicity):
        print(j)
        count = 0
        for i in range(len(CL_train[CL_train.Author==name])):
            if text in CL_train[CL_train.Author==name].iloc[i,5]:
                count += 1


        conta.append(count)
    fake[name] = conta

Juan Williams
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
Bill Moyers
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
Paul Krugman
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
Joseph E. Stiglitz
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [99]:
for name in CL_train.Author.unique():
    print(name)
    CL_train.loc[CL_train.Author==name, 'rep'] = fake[name]


Juan Williams
Bill Moyers
Paul Krugman
Joseph E. Stiglitz
Jesse Jackson
George Soros
Donna Brazile
James Carville
Tony Judt
Jane Jacobs
Lawrence Lessig
Glenn Greenwald
Al Gore
Alan Bennett
Anthony Giddens
Benjamin Barber


In [100]:
CL_train.rep.value_counts()

1.0     1647
2.0       21
3.0        5
5.0        1
10.0       1
12.0       1
Name: rep, dtype: int64

In [101]:
CL_train.to_csv('CL_train_clean.csv', index=False)

In [102]:
CL_test.Author.unique()

array(['Robert Reich', 'Nicholas Kristof', 'Polly Toynbee', 'Ezra Klein'],
      dtype=object)

In [103]:
autoref_test=find_autoref(CL_test)

In [104]:
CL_test['auto'] = 0
for name in CL_test.Author.unique():
    print(name, ' done')
    CL_test.loc[CL_test.Author==name, 'auto'] = autoref_test[name]

Robert Reich  done
Nicholas Kristof  done
Polly Toynbee  done
Ezra Klein  done


In [105]:
CL_test.auto.sum()

0

In [107]:
CL_test

,Author,Quotes,Contain,Political Orientation,auto
0,Robert Reich,"Globalization and free trade do spur economic growth, and they lead to lower prices on many goods.",no,Progressive,0
1,Robert Reich,True patriotism isn't cheap. It's about taking on a fair share of the burden of keeping America going.,no,Progressive,0
2,Robert Reich,"The faith that anyone could move from rags to riches - with enough guts and gumption, hard work and nose to the grindstone - was once at the core of the American Dream.",no,Progressive,0
3,Robert Reich,"The only way to grow the economy in a way that benefits the bottom 90 percent is to change the structure of the economy. At the least, this requires stronger unions and a higher minimum wage.",no,Progressive,0
4,Robert Reich,You can't inspire people if you are going to be uninspiring.,no,Progressive,0
5,Robert Reich,One tax dodge often used by multi-national companies is to squirrel their earnings abroad in foreign subsidiaries located in countries where taxes are lower.,no,Progressive,0
6,Robert Reich,"Bankruptcy laws allow companies to smoothly reorganize, but not college graduates burdened by student loans.",no,Progressive,0
7,Robert Reich,"As public schools deteriorate, the upper-middle class and wealthy send their kids to private ones. As public pools and playgrounds decay, the better-off buy memberships in private tennis and swimming clubs. As public hospitals decline, the well-off pay premium rates for private care.",no,Progressive,0
8,Robert Reich,"As income from work has become more concentrated in America, the super rich have invested in businesses, real estate, art, and other assets. The income from these assets is now concentrating even faster than income from work.",no,Progressive,0
9,Robert Reich,"What are called 'public schools' in many of America's wealthy communities aren't really 'public' at all. In effect, they're private schools, whose tuition is hidden away in the purchase price of upscale homes there, and in the corresponding property taxes.",no,Progressive,0


In [108]:
CL_test=CL_test.drop(CL_test.index[[4,32,88]])

In [109]:
CL_test['texicity'] = CL_test.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True,
                                                                     no_numbers=True))

In [110]:
CL_test = tok_stop(CL_test)

- 

In [115]:
CL_test.to_csv('CL_test_clean.csv', index=False)

In [112]:
fake = {}
#count = 0
for name in CL_test.Author.unique():
    print(name)
    conta = []
    for i,text in enumerate(CL_test[CL_test.Author==name].texicity) :
        print(i)
        count = 0
        for i in range(len(CL_test[CL_test.Author==name])):
            if text in CL_test[CL_test.Author==name].iloc[i,5]:
                count += 1


        conta.append(count)
    fake[name] = conta

Robert Reich
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
Nicholas Kristof
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
Polly Toynbee
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Ezra Klein
0
1
2
3
4
5
6
7
8
9
10
11


In [113]:
CL_test['rep'] = 0
for name in CL_test.Author.unique():
    print(name, ' done')
    CL_test.loc[CL_test.Author==name, 'rep'] = fake[name]


Robert Reich  done
Nicholas Kristof  done
Polly Toynbee  done
Ezra Klein  done


In [114]:
CL_test.rep.value_counts()

1    90
Name: rep, dtype: int64

In [116]:
CL_train.to_csv('CL_train_clean.csv')